In [ ]:
!pip install requests gql python-dotenv websockets

In [ ]:
import os
import csv
from dotenv import load_dotenv
import requests
import json
import asyncio
from gql import Client, gql
from gql.transport.websockets import WebsocketsTransport

load_dotenv()

In [ ]:
access_token = os.getenv("BITQUERY_ACCESS_TOKEN")
mint_address = os.getenv("MINT_ADDRESS")

In [ ]:
import asyncio
import csv
from gql import gql
from gql.transport.websockets import WebsocketsTransport
from websockets.exceptions import ConnectionClosedError

async def subscribe_and_save(transport, query, csv_file):
    try:
        async for result in transport.subscribe(query):
            data = result.data.get('Solana', {}).get('DEXTradeByTokens', [])
            rows = []
            for trade in data:
                time = trade.get('Block', {}).get('Time')
                price = trade.get('Trade', {}).get('PriceInUSD')
                if time and price is not None:
                    print(f"Time: {time}, PriceInUSD: {price}")
                    rows.append([time, f"{price:.10f}"])

            # Append the new data to the CSV file
            if rows:
                with open(csv_file, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerows(rows)
    except ConnectionClosedError as e:
        print(f"Connection closed: {e}")
        raise  # Re-raise to trigger the reconnect logic
    except asyncio.CancelledError:
        print("Subscription cancelled.")

async def main():
    csv_file = "price_data.csv"

    # Initialize the CSV file with headers if it doesn't exist
    try:
        with open(csv_file, 'x', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Time", "PriceInUSD"])  # Write headers
    except FileExistsError:
        pass  # File already exists, no need to write headers

    query = gql(f"""
      subscription MyQuery {{
        Solana {{
        DEXTradeByTokens(
          orderBy: {{descending: Block_Time}}
          limit: {{count: 100}}
          where: {{
          Trade: {{
            Dex: {{ProgramAddress: {{is: "6EF8rrecthR5Dkzon8Nwu78hRvfCKubJ14M5uBEwF6P"}}}},
            Currency: {{MintAddress: {{is: "{mint_address}"}}}}
          }},
          Transaction: {{Result: {{Success: true}}}}
          }}
        ) {{
          Block {{
          Time
          }}
          Trade {{
          PriceInUSD
          }}
        }}
        }}
      }}
    """)

    retries = 0
    max_retries = 5  # Set a maximum retry limit if needed

    while retries < max_retries:
        transport = WebsocketsTransport(
            url=f"wss://streaming.bitquery.io/eap?token={access_token}",
            headers={"Sec-WebSocket-Protocol": "graphql-ws"}
        )

        try:
            # Connect to the WebSocket
            await transport.connect()
            print("Connected")

            # Run the subscription with a timeout
            await asyncio.wait_for(subscribe_and_save(transport, query, csv_file), timeout=100)

        except (ConnectionClosedError, asyncio.TimeoutError) as e:
            print(f"Error occurred: {e}. Retrying...")
            retries += 1
            await asyncio.sleep(5)  # Wait before retrying

        except Exception as e:
            print(f"Unexpected error: {e}")
            break

        finally:
            # Ensure the transport is closed before retrying
            await transport.close()
            print("Transport closed")

    if retries >= max_retries:
        print("Max retries reached. Exiting.")

try:
    await main()
except RuntimeError as e:
    print(f"Runtime error: {e}")
